# Modelado para predicción de sentimientos

Esta modelación parte a partir de los datos que han sido extraidos vía Twitter y han sido limpiados a traves del cleaning presentado en la 02-Limpieza

In [ ]:
# Lectura de librerias

library(twitteR)
library(twitteR)
library(ROAuth)
library(plyr)
library(dplyr)
library(stringr)
library(ggplot2)
library(devtools)
library(tidyverse)
library(text2vec)
library(caret)
library(glmnet)
library(ggrepel)

In [ ]:
## Cargando base de datos de tweets para entrenamiento

tweets_classified <- read_csv('dataset_training.csv',
                              col_names = c('sentiment', 'id', 'date', 'query', 'user', 'text')) %>%
  # converting some symbols
  dmap_at('text', conv_fun) %>%
  # replacing class values
  mutate(sentiment = ifelse(sentiment == 0, 0, 1))

In [ ]:
## Limpieza de NA

tweets_classified_na <- tweets_classified %>%
 filter(is.na(id) == TRUE) %>%
 mutate(id = c(1:n()))


tweets_classified <- tweets_classified %>%
 filter(!is.na(id)) %>%
 rbind(., tweets_classified_na)

In [ ]:
# Partición de dato

# Inicialización semilla para resultados reproducibles
set.seed(2340)


trainIndex <- createDataPartition(tweets_classified$sentiment, p = 0.8, 
                                  list = F, 
                                  times = 1)


# Partición del dataser
tweets_train <- tweets_classified[trainIndex, ]
tweets_test <- tweets_classified[-trainIndex, ]


# Tokenización de los tweets
prep_fun <- tolower
tok_fun <- word_tokenizer

it_train <- itoken(tweets_train$text, 
                   preprocessor = prep_fun, 
                   tokenizer = tok_fun,
                   ids = tweets_train$id,
                   progressbar = TRUE)

it_test <- itoken(tweets_test$text, 
                  preprocessor = prep_fun, 
                  tokenizer = tok_fun,
                  ids = tweets_test$id,
                  progressbar = TRUE)

# Vocacbulario
vocab            <- create_vocabulary(it_train)
vectorizer       <- vocab_vectorizer(vocab)
dtm_train        <- create_dtm(it_train, vectorizer)
dtm_test         <- create_dtm(it_test, vectorizer)

tfidf            <- TfIdf$new()

# Ajuste modelo y transformación
dtm_train_tfidf  <- fit_transform(dtm_train, tfidf)
dtm_test_tfidf   <- fit_transform(dtm_test, tfidf)


In [ ]:
## Entrenamiento del modelo
t1 <- Sys.time()

glmnet_classifier <- cv.glmnet(x = dtm_train_tfidf, 
                               y = tweets_train[['sentiment']], 
                               family = 'binomial', 
                               alpha = 1,
                               type.measure = "auc", # Area bajo la curca ROC
                               nfolds = 5,
                               thresh = 1e-3,
                               maxit = 1e3)
t2 <- Sys.time()
t2-t1 # Tiempo de ejecución del entrenamiento

In [ ]:
# Graficas de modelación
options(repr.plot.width=10, repr.plot.height=4)
print(difftime(Sys.time(), t1, units = 'mins'))

plot(glmnet_classifier)
print(paste("max AUC =", round(max(glmnet_classifier$cvm), 4)))

In [ ]:
# Predicciones del modelo
preds <- predict(glmnet_classifier,
                 dtm_test_tfidf, 
                 type = 'response')[ ,1]

glmnet:::auc(as.numeric(tweets_test$sentiment), preds)

In [ ]:
# Guardar los resultados del modelo ajustado
saveRDS(glmnet_classifier, 'glmnet_classifier.RDS')

# Graficando nube de palabras

In [ ]:
## Generación de nube de datos - Wordcloud
library(wordcloud)

corpus       <- Corpus(VectorSource(corpus))
corpus       <- tm_map(corpus, toSpace, x)

#convierte el corpus en un documento-elemento matriz
tdm         <- TermDocumentMatrix(corpus,
                                  control = list(removePunctuation = TRUE,
                                                 stopwords = c(stopwords("spanish"), stopwords("es"), x),
                                                 removeNumbers = TRUE, tolower = TRUE))

# Convierte el documento en un formato matrix
m = as.matrix(tdm)

# get word counts in decreasing order
word_freqs <- sort(rowSums(m), decreasing=TRUE)

# create a data frame with words and their frequencies
dm         <- data.frame(word=names(word_freqs), freq=word_freqs)

# plot wordcloud
options(repr.plot.width=7, repr.plot.height=7)

wordcloud(dm$word, dm$freq, random.order=FALSE, colors=brewer.pal(8, "Dark2"))
lapply(data, function(x) write.table( data.frame(x), 'test.csv'  , append= T, sep=',' ))

# Fuente

Esta modelación y código se baso en la siguiente publicación:

https://www.r-bloggers.com/twitter-sentiment-analysis-with-machine-learning-in-r-using-doc2vec-approach/